## **SpDB Manual之系列2：处理Python中原生数据类型**

### 提供统一的Entry入口。
- Entry为数据对象提供一个指针游标，并不真正的获取数据，只有当执行get操作时，数据才被真正访问。
- 其对象可以是Dict，也可以是List

In [127]:
## 加载基本环境
from spdm.data.Entry import open_entry,Entry
from spdm.utils.logger import logger

In [231]:
### 接收字典
example_dict = {
    'name': 'Alice',
    'age': 25,
    'hobbies': ['reading', 'painting', 'yoga'],
    'address': [
        {
            'street': '123 Main St',
            'city': 'Anytown',
            'state': 'CA',
            'zip': '12345'
        },
        {
            'street': '456 Oak St',
            'city': 'Othertown',
            'state': 'NY',
            'zip': '67890'
        },
        {
            'street': '789 Elm St',
            'city': 'Somewhere',
            'state': 'CO',
            'zip': '24680'
        }
    ],
    'spouse': {
        'spouse_name':
            {
                'name':'Bob',
                'age': 27,
            },
        'name': 'Bob',
        'age': 27,
        'hobbies': ['music', 'skiing', 'reading']
    }
}
dict_entry = Entry(data=example_dict)
dict_entry.get()

{'name': 'Alice',
 'age': 25,
 'hobbies': ['reading', 'painting', 'yoga'],
 'address': [{'street': '123 Main St',
   'city': 'Anytown',
   'state': 'CA',
   'zip': '12345'},
  {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
  {'street': '789 Elm St',
   'city': 'Somewhere',
   'state': 'CO',
   'zip': '24680'}],
 'spouse': {'spouse_name': {'name': 'Bob', 'age': 27},
  'name': 'Bob',
  'age': 27,
  'hobbies': ['music', 'skiing', 'reading']}}

In [186]:
### 接收list
my_list = [
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13],
    [14, 15, 16, 17],
    [18, 19, 20, 21, 22, 23, 24]
]      
list_entry = Entry(my_list)
list_entry.get()

[[1, 2, 3, 4, 5],
 [6, 7, 8, 9, 10],
 [11, 12, 13],
 [14, 15, 16, 17],
 [18, 19, 20, 21, 22, 23, 24]]

### Enry操作支持指标游走：
- child():将指标移动到目标子节点
- paraent():将指标移动到目标父节点
- next():将指标移动到目标节点内元素的兄弟
 - 该功能只对元素是List有效

In [219]:
### 指标游走到当前节点"spouse.spouse_name"
child_data = dict_entry.child("spouse.spouse_name")
child_data.get()

{'name': 'Bob', 'age': 27}

In [221]:
### 指标移到当前节点的paraent节点，并获取数据
paraent_tree = child_data.parent
paraent_tree.get()

{'spouse_name': {'name': 'Bob', 'age': 27},
 'name': 'Bob',
 'age': 27,
 'hobbies': ['music', 'skiing', 'reading']}

In [238]:

dict_entry.child("address[0]").parent.get()

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [232]:
### 当当前节点的元素是LIST时，指标可以移动遍历到其节点内所有value
print(dict_entry.child("address[0]").next().get())
print(dict_entry.child("address[1]").next().get())


{'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'}
{'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}


### 在Entry操作的基础上，可灵活支持数据读写和更新：
- get():获取指标所指向的数据
- put():更新指标所指向的数据
- insert():在指标所指向的数据中插入数据
- update():在指标所指向的数据中更新数据
- for_each():在指标所指向的数据中遍历数据
- keys():在指标所指向的数据中查找数据的键

In [239]:
### 对象是dict，get所有数据
dict_entry.get()

{'name': 'Alice',
 'age': 25,
 'hobbies': ['reading', 'painting', 'yoga'],
 'address': [{'street': '123 Main St',
   'city': 'Anytown',
   'state': 'CA',
   'zip': '12345'},
  {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
  {'street': '789 Elm St',
   'city': 'Somewhere',
   'state': 'CO',
   'zip': '24680'}],
 'spouse': {'spouse_name': {'name': 'Bob', 'age': 27},
  'name': 'Bob',
  'age': 27,
  'hobbies': ['music', 'skiing', 'reading']}}

In [241]:
## get某个子节点的数据
dict_entry.child("spouse").get()

{'spouse_name': {'name': 'Bob', 'age': 27},
 'name': 'Bob',
 'age': 27,
 'hobbies': ['music', 'skiing', 'reading']}

In [244]:
### 对象是list。get得到所有数据
list_entry.get()


[[1, 2, 3, 4, 5],
 [6, 7, 8, 9, 10],
 [11, 12, 13],
 [14, 15, 16, 17],
 [18, 19, 20, 21, 22, 23, 24]]

In [246]:
### List对象标识符以数字编号排序，get得到第4个子节点的数据
list_entry.child(4).get()

[18, 19, 20, 21, 22, 23, 24]

In [247]:
### for_each用来遍历dict中delist
for v in dict_entry.child("address").for_each():
    print(v)    

(0, {'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'})
(1, {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'})
(2, {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'})


In [248]:
### insert插入增加一个值，使之前的数变成list
dict_entry.child("age").insert(50)
dict_entry.child("age").get()

[25, 50]

In [250]:
dict_entry.child("name").insert({"first":"Alice","last":"Smith"})
## 注意，插入更多数据后，原来的数据被组成了LIST
dict_entry.child("name").get()

['Alice', {'first': 'Alice', 'last': 'Smith'}]

In [254]:
### update是更新之前的整个数值
print(f'原来的age是',dict_entry.child("age").get())
dict_entry.child("age").update(100)
print(f'现在的age是',dict_entry.child("age").get())
# dict_entry.child("age").get()

原来的age是 90
现在的age是 100


In [275]:
### 计算当前节点的元数个数
print(f'address的元数个数是',dict_entry.child("address").count)
print(f'子节点spouse.spouse_name的元数个数是',dict_entry.child("spouse.spouse_name").count)

address的元数个数是 3
子节点spouse.spouse_name的元数个数是 1


In [ ]:
### 获得节点的关键keys
dict_entry.keys()